In [1]:
%reload_ext Cython

In [2]:
#%%cython
import typing
import copy
import pdb
from IPython.display import clear_output
import numpy as np
import time
from colorama import Fore
import sys
import cProfile
import multiprocessing
import chess 
import board

ModuleNotFoundError: No module named 'board'

In [ ]:
all_squares = [[i,j] for i in range(8) for j in range(8)]
all_moves = [i+j for i in all_squares for j in all_squares]
pieces = ["Empty","Rook","Knight","Bishop","Queen","King","Pawn","Pawn","King","Queen","Bishop","Knight","Rook"]

piece_owner = [0,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1]
list_letters = ['a','b','c','d','e','f','g','h']
list_numbers = list(map(str,list((range(1,9)))))
legal_outputs = ["pawn_move","pawn_move,en-passant","king_move",'king_move,castling,kingside','king_move,castling,queenside','en-passant','']
en_passant = ["c7c5","a2a3","c5c4","b2b4","c4b3"]
zero_line = [0 for _ in range(8)]

In [ ]:
#%%cython
#Creates a new board
def create_board(): 
    #pdb.set_trace()
    val = 0
    board = np.array([[val for j in range(8)] for i in range(8)])
    for i in range(8):
        board[1,i] = 6
        board[6,i] = -6
        
    board[0,4] = 5
    board[7,4] =  -5

    board[0,1] = 2
    board[7,1] = -2

    board[0,6] = 2
    board[7,6] = -2

    board[0,2] = 3
    board[7,2] = -3

    board[0,5] = 3 
    board[7,5] = -3

    board[0,3] = 4
    board[7,3] = -4

    board[0,4] = 5
    board[7,4] = -5
    
    board[0,0] = 1
    board[7,0] = -1 
    
    board[0,7] = 1
    board[7,7] = -1
    return [board]

def create_rights():
    rights = np.array([[0 for j in range(8)] for i in range(8)])
    rights[0,4] = 1
    rights[7,4] = -1
    
    rights[0,0] = 1
    rights[7,0] = -1
    rights[0,7] = 1
    rights[7,7] = -1
    return [rights]

def create_player_positions():
    positions_1 = []
    for i in range(8):
        positions_1.append([0,i])
        positions_1.append([1,i])
    
    positions_2 = []
    for i in range(8):
        positions_2.append([6,i])
        positions_2.append([7,i])
        
    return [[positions_1] + [positions_2]]
    
def create_chess_status():
    return [False]

def create_king_positions():
    king_positions = [[0,4],[7,4]]
    return [king_positions]

def get_from_square_player(i,j,board):
    val = board[i,j]
    return val / abs(val)


def pp_board(board):
    for i in range(8):
        print(Fore.BLACK + list_numbers[7-i],end="\t")
        for j in range(8):
            player = piece_owner[board[i,j]]
            if player == 0:
                print(Fore.BLACK + "_",end="\t")
            else: 
                piece = pieces[board[i,j]]
                if player== 1:
                    print(Fore.BLUE + piece,end="\t")
                else:
                    print(Fore.BLACK + piece,end="\t")
        print("\n")
    print("",end="\t")
    for i in list_letters:
        print(Fore.BLACK + i,end="\t")
    sys.stdout.flush()

In [ ]:
class ChessBoard:
    def __init__(self,player):
        self.board = create_board()
        self.rights = create_rights()
        self.king_positions = create_king_positions()
        self.player_positions = create_player_positions()
        self.chess_status = create_chess_status()
        self.player = player
        self.move_status = ""
    
    def make_move(self,move):
        self.board,self.move_status,self.rights,self.player,self.king_positions,self.player_positions,self.chess_status = board.make_move(self.board,
                                                                                                                                          move,
                                                                                                                                          self.rights,
                                                                                                                                          self.player,
                                                                                                                                          self.king_positions,
                                                                                                                                          self.player_positions,
                                                                                                                                          self.chess_status)
    
    def unmake_move(self,move):
        self.board,self.move_status,self.rights,self.player,self.king_positions,self.player_positions,self.chess_status = board.unmake_move(self.board,
                                                                                                                                            self.rights,
                                                                                                                                            self.player,
                                                                                                                                            self.king_positions,
                                                                                                                                            self.player_positions,
                                                                                                                                            self.chess_status)
    def show_board(self):
        pp_board(self.board[-1])
    
    def generate_legal_moves(self):
        return board.generate_all_legal_moves(self.board,self.rights,self.player,
                                        self.king_positions,self.player_positions,
                                        self.chess_status,board.all_moves)
        

In [ ]:
b = ChessBoard(1)
b.show_board()
b.generate_legal_moves()

In [ ]:
#%%cython
def curate_pgn_string(s):
    s = s.replace("\n"," ")
    s = s.replace("+","")
    s = s.replace("Q","")
    s = s.replace("N","")
    s = s.replace("-","")
    s_list = s.split(' ')
    s_list
    list_remove = [s_list[i] for i in range(len(s_list)) if i%3 == 0]
    for i in list_remove:
        s_list.remove(i)
    try:
        s_list.remove("")
    except:
        s_list = s_list
    return s_list

path = "extracted/two.txt"
f = open(path,"r")
s = f.read()
curated = curate_pgn_string(s)

In [ ]:
#Test cell. Here are the functions that test the engine
def perft_test(depth):
    b = create_board()
    r = create_rights()
    k = create_king_positions() 
    p_pos = create_player_positions()
    chess = create_chess_status()
    p = 1
    start = time.time()
    val = perft(b,r,p,k,p_pos,chess,all_moves,depth)
    end = time.time() 
    return val,end-start

def wrong_moves(depth):
    #correct moves
    board = chess.Board()
    legal_moves = list(map(board.lan,board.legal_moves))
    s = ""
    for i in legal_moves:
        s = s + i + ' '
    print(legal_moves)
    s_curated = curate_pgn_string(s)
    print(s_curated)
    #your moves
    
    #diff

#cProfile.run('perft_test(2)')
wrong_moves(5)

In [ ]:
#perft_test(2)

In [ ]:
#%%cython
def play_game_until_error(curated_pgn):
    b = ChessBoard(-1)
#    b = create_board()
#    r = create_rights()
#    k = create_king_positions() 
#    player_pos = create_player_positions()
#    chess = create_chess_status()
#    s = ""
#    p = -1
    count = 0
    for move in curated_pgn:
        #if count == 3:
            #pdb.set_trace()
        b.make_move(move)
        
#        b,s,r,p,k,player_pos,chess= make_move(b,move,r,p,k,player_pos,chess)
        count += 1
        if not(b.move_status in legal_outputs):#== "castling,k" or s == "castling,q" or s == "" or s == "en-passant"):
            print(count,s)
            print(move)
            board.show_board()
            #pp_board(b[-1])
            break
    success = (b.move_status in legal_outputs)# == "castling,k" or s == "castling,q" or s == "" or s == "en-passant")
    return count, move,success,s
play_game_until_error(curated)

In [ ]:
#%%cython
def start_game(player,moves=[]):
    #pdb.set_trace()
    b = create_board()
    r = create_rights()
    k = create_king_positions()
    player_pos = create_player_positions()
    chess = create_chess_status()
    s = ""
    p = player 
    count = 0
    do = "m"
    i = 0
    prev = "m"
    watching = len(moves) != 0
    stop = False
    pp_board(b[-1])


    while not stop:
        do = input()
#        if do == "":
#            do = prev
#            
        if watching:
            if i == len(moves):
                stop = True
                
            if do == "m":
                b,s,r,p,k,player_pos,chess = make_move(b,moves[i],r,p,k,player_pos,chess)
                prev == "m"
                
                print(moves[i])
            
            elif do == "u":
                b,s,r,p,k,player_pos,chess = unmake_move(b,r,p,k,player_pos,chess)
                if s == "n":
                    continue
                i = i-2
                prev = "u"
                
                print(moves[i])
            elif do == "exit":
                stop = True
            else:
                print("wrong input")
                continue
        else:  
            if do == "u":
                b,s,r,p,k,player_pos,chess = unmake_move(b,r,k,player_pos,chess)
                if s == "n":
                    print("didn't unmake")
                    continue
                i = i-2
                continue
            
            else:
                b,s,r,p,k,player_pos,chess = make_move(b,do,r,p,k,player_pos,chess)
                if not(s in legal_outputs):
                    print("wrong input")
                    print(s)
                    continue

        clear_output()
        pp_board(b[-1])
        print(s)

        sys.stdout.flush()
        count += 1
        i = i + 1
    clear_output()
    pp_board(b[-1])
start_game(-1,curated)